<p align="center">
  <img src="https://media.istockphoto.com/id/1135912032/photo/respiratory-system.jpg?s=2048x2048&w=is&k=20&c=9vBNs47uX_gaY32HKTzgO4t2I-gaYIV3LA7xZAtM1CA=" style="width:100%; max-width:900px; height:180px; object-fit:cover; border-radius:10px;"/>
</p>
<div style="text-align:center;">
  <h1 style="color:Gray; display:inline-block; margin:0;; size:2.5em">Lung Life</h1><br>
  <h1 style="color:Gray; display:inline-block; margin:0;">Aplicacion movil para diagnostico temprano de cancer pulmonar</h1>
  <p>
    <b>LungLife | Machine Learning | CRISP-DM</b><br>
    <span style="font-size:1.1em;">Análisis y predicción de la salud pulmonar utilizando machine learning.</span>
  </p>
</div>

# FASE 3: Preparación de Datos (CRISP-DM)

## Propósito de la Fase de Preparación de Datos

En esta tercera fase del proceso CRISP-DM, nos enfocamos en preparar y transformar los datos para que sean adecuados para el modelado. Esto incluye la limpieza de datos, la selección de características relevantes y la creación de nuevas variables que puedan mejorar el rendimiento del modelo. El objetivo es asegurar que los datos estén en la mejor forma posible para entrenar modelos predictivos efectivos.

In [ ]:
# Verifica si las librerías necesarias ya están instaladas
try:
    # Ignorar warnings
    import warnings
    warnings.filterwarnings('ignore')
    # Librerías del sistema
    import sys, subprocess
    # Importaciones base
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    import os as os  
    from os import path
    import joblib
    import scipy.stats as stats
    import pickle as pkl

    print("Dependencias ya instaladas.")

except ImportError:

    print("Dependencias no encontradas. Instalando ahora...")
    
    # Se ejecutan los comandos de instalación
    %pip install --quiet matplotlib
    %pip install --quiet seaborn
    %pip install --quiet joblib
    %pip install --quiet scipy
    
    print("Instalación completada.")

In [ ]:
# Cargar datasets 
df = pd.read_csv('../data/raw/lung-cancer-structured-clinical-dataset/lung_cancer_data.csv')  # Ejemplo de carga de dataset
print("Dataset cargado exitosamente.")

In [ ]:
# Define el nombre del archivo
dataunderstanding = 'dataunderstand_lungcancer.parquet'

# Guarda el DataFrame  en formato Parquet
try:
    df.to_parquet(dataunderstanding, index=False) 
    print(f"DataFrame guardado exitosamente como '{dataunderstanding}'")
except Exception as e:
    print(f"Error al guardar como Parquet: {e}")